In [14]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


# Cases, fine age split

In [15]:

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
latestdir

dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')
dfAge['Nyindlagte pr. 100.000 borgere'] = pd.to_numeric(dfAge['Nyindlagte pr. 100.000 borgere'].str.replace(',','.'))

dfAge.tail(18)


,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive
814,2021-W45,0-2,4653,159.0,4.9,8535,291.0
815,2021-W45,3-5,17945,386.0,NaN,33586,723.0
816,2021-W45,6-11,30918,1326.0,1.6,113836,4881.0
817,2021-W45,12-15,22934,463.0,1.1,62512,1263.0
818,2021-W45,16-19,15672,295.0,1.8,42942,808.0
819,2021-W45,20-39,17853,396.0,4.5,264478,5865.0
820,2021-W45,40-64,22248,347.0,6.2,420588,6565.0
821,2021-W45,65-79,11491,202.0,15.9,103764,1826.0
822,2021-W45,80+,6685,129.0,32.9,19519,378.0
823,2021-W46,0-2,5527,199.0,10.9,10139,365.0


In [16]:
curYears = dfAge.Uge.apply(lambda x: int(x[:4]))

# Remove everything before 2021
curdf = dfAge[curYears > 2020].copy()


weekNums = curdf.Uge.apply(lambda x: int(x[-2:]))


# Uge 36: 07-09-2021 (tirsdag) og 12-09-2021 (søndag)

weekOffset = weekNums - 36
# allDates = weekOffset.apply(lambda x:np.datetime64('2021-09-07') + np.timedelta64(x*7,'D'))
allDates = weekOffset.apply(lambda x:np.datetime64('2021-09-12') + np.timedelta64(x*7,'D'))

allDates
curdf['Dato'] = allDates

In [17]:


# fig,ax1 = plt.subplots()
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal positive'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal positive'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)
    toAdd = thisdf['Antal positive'].values
    toAdd = thisdf['Antal positive'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,label=curAge)
    ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Nye smittetilfælde\n[ugentligt gennemsnit, per dag]')
ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_Cases')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:


# fig,ax1 = plt.subplots()
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal positive'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal positive'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)
    toAdd = thisdf['Antal positive'].values
    # toAdd = thisdf['Antal positive'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentligt nye smittetilfælde')
ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_CasesWeekly')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:


fig,ax1 = plt.subplots()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
# ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal positive'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal positive'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('tab20b',len(allAgeGroups)+1)
# cmap = plt.cm.get_cmap('inferno',len(allAgeGroups)+5)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)
    toAdd = thisdf['Positive pr. 100.000 borgere'].values
    # toAdd = thisdf['Antal positive'].values/7
    # curCount = curCount + toAdd
    ax1.plot(thisdf.Dato,toAdd,'.-',color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    # ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentligt nye smittetilfælde per 100.000')
# ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
# ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_CasesWeeklyPer100000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Testede

In [20]:
curdf.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Dato
827,2021-W46,16-19,16085,405.0,0.7,44075,1109.0,2021-11-21
828,2021-W46,20-39,19515,486.0,5.7,289097,7201.0,2021-11-21
829,2021-W46,40-64,23987,437.0,7.7,453464,8268.0,2021-11-21
830,2021-W46,65-79,10894,252.0,15.5,98376,2279.0,2021-11-21
831,2021-W46,80+,6270,155.0,42.8,18308,452.0,2021-11-21


In [21]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal testede'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal testede'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal testede'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal testede'],label=curAge)
    toAdd = thisdf['Antal testede'].values
    toAdd = thisdf['Antal testede'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,label=curAge)
    ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Testede \n[ugentligt gennemsnit, per dag]')
ax2.set_ylabel('Andel af alle testede')


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_Testede')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
curdf.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Dato
827,2021-W46,16-19,16085,405.0,0.7,44075,1109.0,2021-11-21
828,2021-W46,20-39,19515,486.0,5.7,289097,7201.0,2021-11-21
829,2021-W46,40-64,23987,437.0,7.7,453464,8268.0,2021-11-21
830,2021-W46,65-79,10894,252.0,15.5,98376,2279.0,2021-11-21
831,2021-W46,80+,6270,155.0,42.8,18308,452.0,2021-11-21


In [23]:


fig,ax1 = plt.subplots()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
# ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Testede pr. 100.000 borgere'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Testede pr. 100.000 borgere'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Testede pr. 100.000 borgere'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('tab20b',len(allAgeGroups)+1)
# cmap = plt.cm.get_cmap('inferno',len(allAgeGroups)+5)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal Testede'],label=curAge)
    toAdd = thisdf['Testede pr. 100.000 borgere'].values
    # toAdd = thisdf['Antal Testede'].values/7
    # curCount = curCount + toAdd
    ax1.plot(thisdf.Dato,toAdd,'.-',color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    # ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentligt testede per 100.000')
# ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
# ax1.set_ylim(top = 25000)
# ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
# ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_TestedeWeeklyPer100000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
print(curdf.tail(9).sum())
display(curdf.tail(9))
print((6324+25698+ 88230)/831807)
print((189.0+522.0+ 3459.0)/15867.0)
# Testede pr. 100.000 borgere

Uge                               2021-W462021-W462021-W462021-W462021-W462021-W...
Aldersgruppe                                 0-23-56-1112-1516-1920-3940-6465-7980+
Testede pr. 100.000 borgere                                                  163943
Positive pr. 100.000 borgere                                                 4566.0
Nyindlagte pr. 100.000 borgere                                                 89.7
Antal testede                                                               1151699
Antal positive                                                              27888.0
dtype: object


,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Dato
823,2021-W46,0-2,5527,199.0,10.9,10139,365.0,2021-11-21
824,2021-W46,3-5,20747,526.0,1.6,38832,984.0,2021-11-21
825,2021-W46,6-11,34895,1558.0,4.1,128476,5737.0,2021-11-21
826,2021-W46,12-15,26023,548.0,0.7,70932,1493.0,2021-11-21
827,2021-W46,16-19,16085,405.0,0.7,44075,1109.0,2021-11-21
828,2021-W46,20-39,19515,486.0,5.7,289097,7201.0,2021-11-21
829,2021-W46,40-64,23987,437.0,7.7,453464,8268.0,2021-11-21
830,2021-W46,65-79,10894,252.0,15.5,98376,2279.0,2021-11-21
831,2021-W46,80+,6270,155.0,42.8,18308,452.0,2021-11-21


0.14456718926385567
0.26280960484023447


In [25]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Nyindlagte pr. 100.000 borgere'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Nyindlagte pr. 100.000 borgere'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Nyindlagte pr. 100.000 borgere'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Nyindlagte pr. 100.000 borgere'],label=curAge)
    toAdd = thisdf['Nyindlagte pr. 100.000 borgere'].values
    toAdd = thisdf['Nyindlagte pr. 100.000 borgere'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,label=curAge)
    ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Nyindlæggelser \n[ugentligt gennemsnit, per dag]')
ax2.set_ylabel('Andel af alle nyindlagte')


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=np.datetime64('2021-07-01'),right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_NyindlagtePer100000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# curdf